In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
MODEL = "llama2"

In [6]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama 

if MODEL.startswith('gpt'):
    model= ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

else:
    model= Ollama(model=MODEL)
model.invoke("Tell me a joke")


"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nHope that made you laugh! Do you want to hear another one?"

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

'The Los Angeles Dodgers won the 2020 World Series, defeating the Tampa Bay Rays in six games (4-2). This took place during the 2020 season, which was impacted by the COVID-19 pandemic. The season and playoffs were condensed into a "bubble" format to minimize the risk of players contracting COVID-19.\n\nThe 2020 postseason was played behind closed doors, with the World Series being played at Globe Life Field in Arlington, Texas (home of the Texas Rangers), which had been designated as a neutral site due to travel restrictions and other safety protocols.'

In [8]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is Susana", question="Who is Mary's sister?")

'Human: \nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Mary\'s sister is Susana\n\nQuestion: Who is Mary\'s sister?\n'

In [10]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is Susana", question="Who is Mary's sister?")

'Human: \nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Mary\'s sister is Susana\n\nQuestion: Who is Mary\'s sister?\n'

In [11]:
chain = prompt | model | parser
chain.invoke({
    "context": "Mary's sister is Susana",
    "question": "Who is Mary's sister?"
})

'Susana'